In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL
from PIL import Image
import random

import numpy as np

import boto3
import sagemaker

import io
import json
from pathlib import Path

from site_tools import category2mask
from matplotlib.backends.backend_pdf import PdfPages
from site_tools import get_matching_s3_keys

profile = 'crayon-site'
region_name='us-east-2'
bucket = 'st-crayon-dev'
prefix = 'sagemaker/labelbox/'

endpoint_name = 'ss-labelbox-I1024-B1024-C896-E20'



In [ ]:
session = boto3.session.Session(profile_name = profile, region_name = region_name)
sess = sagemaker.Session(session,default_bucket=bucket)
s3 = session.resource('s3', region_name=region_name)
bucket_obj = s3.Bucket(bucket)
runtime_client = session.client('runtime.sagemaker')

In [ ]:
def compute_miou(actual, pred, num_cls=6):
    
    print('inside function')

    a = actual
    print(np.unique(a))
    a = a.flatten()
    a_count = np.bincount(a, weights = None, minlength = num_cls) # A
    # print(f'a_count shape = {a_count.shape}')
    
    b = pred
    print(np.unique(a))
    b = b.flatten()
    b_count = np.bincount(b, weights = None, minlength = num_cls) # B
    # print(np.unique(b_count))
    # print(f'b_count shape = {b_count.shape}')

    c = a * num_cls + b
    cm = np.bincount(c, weights = None, minlength = num_cls**2)
    cm = cm.reshape((num_cls, num_cls))

    Nr = np.diag(cm) # A ⋂ B
    Dr = a_count + b_count - Nr # A ⋃ B
    individual_iou = Nr/Dr
    miou = np.nanmean(individual_iou)

    print(miou)
    return individual_iou

In [ ]:
s3_paths = get_matching_s3_keys('st-crayon', 'TestSite/ri4386_chicagoil_DBM6891680292/Mapping/', '.JPG')
s3_paths = [path for path in s3_paths]

In [ ]:
s3_paths = get_matching_s3_keys('st-crayon-dev', 'data/raw/imgs/original/testing/anno_testsite_fcn-512-p2-8xlarge', '.png')
s3_paths = [path for path in s3_paths]

In [ ]:
image_paths = [f"TestSite/ri4386_chicagoil_DBM6891680292/Mapping/{Path(path).stem}.JPG" for path in s3_paths]


In [ ]:
manifest_filepath = 'data/raw/imgs/original/manifests/manifest_file_test_imgs.json'
manifest_file = bucket_obj.Object(key=manifest_filepath).get()
manifest_file = manifest_file['Body'].read().decode('ascii')
j_content = json.dumps(manifest_file)
j_content = json.loads(j_content)

manifest_list = []
for line in j_content.splitlines():
     line = json.loads(line)
     manifest_list.append(line)

source_ref = [f['source-ref'].replace(f's3://{bucket}/','') for f in manifest_list]
annontation_ref = [f['annotation-ref'].replace(f's3://{bucket}/','') for f in manifest_list]

# source_ref_samp = source_ref[:10]
# annontation_ref_sam = annontation_ref[:10]

all_img = source_ref + annontation_ref
print(f'total number of image to be copied = {len(all_img)}')

In [ ]:
endpoint_name = 'psp_1024_old'
endpoint_name = 'psp-1024-new'
endpoint_name = 'fcn-512-new-chip512'
endpoint_name = 'psp-1024-new'
endpoint_name = 'testsite_fcn-512'
pdf_file_name = f'{endpoint_name}.pdf'
num_img_per_pg = 4

In [ ]:
image_refs = [path.replace('/images/', '/testing/images/').replace('.JPG', '.jpg') for path in source_ref]
anno_refs = [path.replace('/annotations/', '/testing/annotations/') for path in annontation_ref]
preds_ref = [path.replace('/annotations/', f'/anno_{endpoint_name}/') for path in anno_refs]

In [ ]:
bucket_obj = s3.Bucket(bucket)
bucket_obj_img = s3.Bucket("st-crayon")
# bucket_obj_img = bucket_obj
bucket_pred = bucket_obj

In [ ]:
image_refs = image_paths
preds_ref = s3_paths

In [ ]:
bucket_obj_img.Object()

In [ ]:
have_ground_truth = False
with PdfPages(pdf_file_name) as pdf:    
    num_images = len(image_refs)
    for i, img_num in enumerate(range(num_images)):
        if (i % num_img_per_pg == 0):
            print('new figure')
            fig, axs = plt.subplots(num_img_per_pg, 3, figsize=(16, num_img_per_pg*6), constrained_layout=True)
        img_count = i % num_img_per_pg
        print(f'processing image-{img_num}-iteration-{i}')
        image_object = bucket_obj_img.Object(image_refs[img_num])
        if have_ground_truth:
            mask_object = bucket_obj.Object(anno_refs[img_num])
        mask_pred = bucket_obj.Object(preds_ref[img_num])
        
        thumbnail_size = 1024
        img_stream = io.BytesIO(image_object.get()['Body'].read())  
        img = Image.open(img_stream)  
        img.thumbnail((thumbnail_size,thumbnail_size), Image.ANTIALIAS)
        if have_ground_truth:
            msk_stream = np.array(Image.open(io.BytesIO(mask_object.get()['Body'].read())))        
            msk = category2mask(msk_stream)
            msk = Image.fromarray(msk, 'RGB')
            msk.thumbnail((thumbnail_size,thumbnail_size), Image.ANTIALIAS)
        msk_pred_stream= np.array(Image.open(io.BytesIO(mask_pred.get()['Body'].read())))        
        msk_pred = category2mask(msk_pred_stream)
        msk_pred = Image.fromarray(msk_pred, 'RGB')
        msk_pred.thumbnail((thumbnail_size,thumbnail_size), Image.ANTIALIAS)
        axs[img_count,0].imshow(img,cmap='gray')
        axs[img_count,0].set_title('orignal image')
        if have_ground_truth:
            axs[img_count,1].imshow(msk,cmap='gray')
            axs[img_count,1].set_title(f'ground truth')
        axs[img_count,2].imshow(msk_pred,cmap='gray')
        axs[img_count,2].set_title(f'prediction')
        if(img_count % num_img_per_pg == num_img_per_pg - 1):
            pdf.savefig()
        elif(num_images - i == 1):
            pdf.savefig()

In [ ]:
for img in all_img:
    p = Path(img)

    file_nam_wo_ext = p.stem
    base_loc = img.split('/')[:-2]
    base_loc = '/'.join(base_loc)

    copy_source = {
    'Bucket': bucket,
    'Key': img
    }
    
    if '/images/' in str(p):
        destination = f'{base_loc}/testing/images/{p.name}'
    elif '/annotations/' in str(p):
        destination = f'{base_loc}/testing/annotations/{file_nam_wo_ext}.png'
    
    # print(f'{copy_source=}')
    try:
        s3.meta.client.copy(copy_source, bucket, destination)
    except:
        print(f'Files not copied :-\n{copy_source=}\n{destination=}')
    # print(f'{destination=}')
print('File copying complete')
    

In [ ]:
%%time
endpoint_name = 'ss-labelbox-I1024-B1024-C896-E20'
random_img_list = random.sample(range(0, len(source_ref)), 10)
# random_img_list =  [64,141,14,122 ]
print(random_img_list)

ious = []


fig, axs = plt.subplots(len(random_img_list), 3, figsize=(16, len(random_img_list)*6), constrained_layout=True)

for i, img_num in enumerate(random_img_list):
    print(f'processing image-{img_num}-iteration-{i}')
    image_object = bucket_obj.Object(source_ref[img_num])
    mask_object = bucket_obj.Object(annontation_ref[img_num])
    
    img_byte_obj = io.BytesIO(image_object.get()['Body'].read())
    img = mpimg.imread(img_byte_obj, 'jpg')
    # mask = mpimg.imread(io.BytesIO(mask_object.get()['Body'].read()), 'png')
    mask_grey = np.array(Image.open(io.BytesIO(mask_object.get()['Body'].read())))
    mask = category2mask(mask_grey)
    
    img_4_pred = Image.open(img_byte_obj)
    # print(type(img_byte_obj))
    # print(type(img_4_pred))

    # print(type(img_4_pred.tobytes()))

    imgByteArr = io.BytesIO()
    img_4_pred.save(imgByteArr, format='PNG')
    imgByteArr = imgByteArr.getvalue()
    # print(type(imgByteArr))

    response = runtime_client.invoke_endpoint(EndpointName = endpoint_name,
                                         ContentType = 'image/jpeg',
                                         Accept = 'image/png',
                                         Body = imgByteArr)
    
    prediction = response['Body'].read() 
    mask_pred_grey = np.array(Image.open(io.BytesIO(prediction)))
    mask_pred = category2mask(mask_pred_grey)

    
    print(np.unique(mask_grey))
    print(np.unique(mask_pred_grey))

    try:
        individual_iou = compute_miou(mask_grey,mask_pred_grey,9)   
        ious.append(individual_iou) 
    except e:
        print(f'An exception occurred + {e}')

    print(ious)


    axs[i,0].imshow(img,cmap='gray')
    axs[i,0].set_title('orignal image')
    axs[i,1].imshow(mask,cmap='gray')
    axs[i,1].set_title(f'image mask \nnumber of classes{np.unique(mask_grey)}')
    axs[i,2].imshow(mask_pred,cmap='gray')
    axs[i,2].set_title(f'predicted mask\n{individual_iou=}\n miou = {str(np.nanmean(individual_iou))}')




# image_object = bucket_obj.Object(source_ref[random_img_num])
# mask_object = bucket_obj.Object(annontation_ref[random_img_num])
# # display(image_object.get())
# img = mpimg.imread(io.BytesIO(image_object.get()['Body'].read()), 'jpg')
# mask = mpimg.imread(io.BytesIO(mask_object.get()['Body'].read()), 'png')

# fig, axs = plt.subplots(1, 2, figsize=(16, 6), constrained_layout=True)
# print(type(axs[0]))
# axs[0].imshow(img,cmap='gray')
# axs[1].imshow(mask,cmap='gray')


In [ ]:

img_orig = '../../../../data/20_AZ_1066_DBM3174640504_-_Mapping_-_DBI0278920773_DBI-21-gnNDUPti-DJI_0374-000006.jpg'
mask_orig = '../../../../data/20_AZ_1066_DBM3174640504_-_Mapping_-_DBI0278920773_DBI-21-gnNDUPti-DJI_0374-000006.png'
# endpoints = sagemaker_client.list_endpoints()

im = PIL.Image.open(img_orig)
# im.thumbnail([800,600],PIL.Image.ANTIALIAS)
# im.save(img_orig, "JPEG")
# converting images to bytearray
with open(img_orig, 'rb') as img_4_pred:
    img_4_pred = img_4_pred.read()
    img_4_pred = bytearray(img_4_pred)
    print(type(img_4_pred))


# envoking the endpoint
response = runtime_client.invoke_endpoint(EndpointName = endpoint_name,
                                         ContentType = 'image/jpeg',
                                         Accept = 'image/png',
                                         Body = img_4_pred)
prediction = response['Body'].read()           
img_orig = PIL.Image.open(img_orig)
mask_orig = category2mask(np.array(PIL.Image.open(mask_orig)))
    
mask_pred = category2mask(np.array(Image.open(io.BytesIO(prediction))))
fig, axs = plt.subplots(1, 3, figsize=(16, 6), constrained_layout=True)
mask = np.array(Image.open(io.BytesIO(prediction)))
axs[0].imshow(img_orig,cmap='gray')
axs[1].imshow(mask_orig,cmap='gray')
axs[2].imshow(mask_pred, cmap='gray')